## 1. Importing Libraries/Tools

In [1]:
# !pip install num2words
# !pip install rake_nltk
# !pip install yake

# from google.colab import drive
# drive.mount('/content/drive')

import datetime
import dateutil
import itertools
import gensim
import gensim.downloader as api
import nltk
import numpy as np
import operator
import pandas as pd
# import rake_nltk
import re
import sklearn
import spacy
import string
import pickle
# import yake

from datetime import datetime
from dateutil.relativedelta import relativedelta
from gensim import corpora
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing
# from gensim.summarization import keywords
# from gensim.summarization.summarizer import summarize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# from num2words import num2words
# from rake_nltk import Rake
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from spacy import displacy
from spacy.tokenizer import Tokenizer

nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wongy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wongy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wongy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\wongy\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wongy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## 2. Load Relevant Data

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/BT4103/Resume.csv')
df1 = df1[['Category','Resume_str']]
df1.Resume_str = df1.Resume_str.apply(lambda x: " ".join(x.split()))

df2 = pd.read_csv('/content/drive/MyDrive/BT4103/Job Description.csv')
df2["job_post_lang"] = df2["job_post_lang"].apply(lambda x: x.lower())
df2 = df2[df2["job_post_lang"] == "en-gb"]
df2 = df2[['category','job_requirements']]
df2 = df2.dropna()

skills_api = pd.read_excel('/content/drive/MyDrive/BT4103/all_skills_emsi.xlsx')
skills_api['name'] = skills_api['name'].apply(lambda x: re.sub("\W?\(.*?\)","",x))
skills_api['name'] = skills_api['name'].apply(lambda x: x.strip().lower())
skills_api = set(skills_api['name'])


## 3. Resume Data Exploration

In [ ]:
df1.Category.value_counts()

INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
FINANCE                   118
ADVOCATE                  118
ACCOUNTANT                118
ENGINEERING               118
CHEF                      118
AVIATION                  117
FITNESS                   117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: Category, dtype: int64

In [ ]:
df1[df1.Category == "INFORMATION-TECHNOLOGY"]

,Category,Resume_str
217,INFORMATION-TECHNOLOGY,INFORMATION TECHNOLOGY Summary Dedicated Infor...
218,INFORMATION-TECHNOLOGY,INFORMATION TECHNOLOGY SPECIALIST GS11 Experie...
219,INFORMATION-TECHNOLOGY,INFORMATION TECHNOLOGY SUPERVISOR Summary Seek...
220,INFORMATION-TECHNOLOGY,INFORMATION TECHNOLOGY INSTRUCTOR Summary Seve...
221,INFORMATION-TECHNOLOGY,INFORMATION TECHNOLOGY MANAGER/ANALYST Profess...
...,...,...
332,INFORMATION-TECHNOLOGY,CORPORATE PROJECT MANAGER Career Overview Seas...
333,INFORMATION-TECHNOLOGY,IT TECHNOLOGY SPECIALIST Professional Summary ...
334,INFORMATION-TECHNOLOGY,IT MANAGER Highlights Customer and Client Rela...
335,INFORMATION-TECHNOLOGY,SUBJECT MATTER EXPERT (INFORMATION TECHNOLOGY ...


In [ ]:
# Test cases
test1 = 'Wong Yung Jer          : (+65) 88870515   : wongyungjer29@gmail.com   : Wong Yung Jer   : Yung-Jer EDUCATION  National University of Singapore (NUS) Aug 2020 – Present • Bachelor of Science in Business Analytics (School of Computing), Year 3  • CAP: 4.70/5.00  • Recipient of ASEAN Undergraduate Scholarship Award  • Dean’s List Student  EXPERIENCE  Undergraduate Student Teaching Assistant, NUS School of Computing  Aug 2022 – Present  • Experienced teaching assistant with a passion for inspiring other students in Mathematics and Artificial Intelligence • Shaped and fostered a conducive classroom environment for free discussion • Conducting weekly tutorials for: • BT2103 Optimization Methods in Business Analytics • BT3017 Feature Engineering for Machine Learning Student Developer, Back-end Web Application Development  Aug 2021 – Present  • Executed the developing, testing, debugging and maintenance of a web system  • Established rest API, routings, input validation testing, documentation with CodeIgniter 3 framework  • Designed a system and helped eliminate up to 95% of manual work to handle form submissions  CO-CURRICULAR ACTIVITIES  Outreach Team Member, NUS Student Union (NUSSU) Committee for Information Technology (commIT)  Sep 2022 – Present  • Collaborating with team-mates to support outreach strategies and promote NUSSU commIT  • Self-initiated a new email automation method and improved efficiency by at least 70%  • Drafted emails and conducted mass email sending campaigns to look for sponsorship for commIT’s flagship events   ACHIEVEMENTS  Certificate of Distinction in the Analytics Techniques Knowledge Area  • Scored an average of at least 4.5 / 5.0 in three modules offered by NUS: Computational Methods for Business Analytics, Econometrics Modelling for Business Analytics and Feature Engineering for Machine Learning  Top Student in Feature Engineering for Machine Learning  • Being awarded the Outstanding Performance for Feature Engineering for Machine Learning in AY 2021/2022 Semester 2  AWS x Carro Hackathon 2022 (4th place)  • Derived a full pipeline with an accuracy of 87% to solve computer vision real world problems • Optimized a sound classification model with an accuracy of 93% to identify faulty car engines KEY SKILLS  Python, R, Java, PHP, Data Mining, Feature Engineering, Data Analysis, Data Visualization (Tableau, R, Matplotlib), Machine Learning Pipeline Design, NLP (BERT, Glove), SQL(MySQL, Oracle SQL), NoSQL (MongoDB), Time Series Analysis (SARIMA, GARCH)  AVAILABILITY PERIOD  • 5 May 2023 – 29 Dec 2023 (3-month summer internship or 6-month internship) '
test2 = df1.iloc[300,1]

Text Pre-processing functions

In [ ]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')
# Tokenize words only with the whitespace rule
# N-grams will no longer be treated as 'N' and '-grams'
nlp.tokenizer = Tokenizer(nlp.vocab, token_match=re.compile(r'\S+').match)

def preprocess(txt):
    txt = txt.lower()
    # these must come first
    txt = re.sub('b\.\S*', '', txt) # remove all bachelor qualifications
    txt = re.sub('m\.\S*', '', txt) # remove all master qualifications
    
    txt = txt.replace("'","").replace("’","") # remove apostrophes
    txt = re.sub("<.*?>"," ",txt) # remove <> tags
    txt = re.sub('http\S+\s*', ' ', txt)  # remove URLs
    txt = re.sub('RT|cc', ' ', txt)  # remove RT and cc
    txt = re.sub('#\S+', '', txt)  # remove hashtags
    txt = re.sub('@\S+', '  ', txt)  # remove mentions
    txt = re.sub('[^a-zA-Z]', ' ', txt) # Remove non-English characters
    txt = re.sub('\s+', ' ', txt)  # remove extra whitespace

    # tokenize word
    txt = nlp(txt)

    # remove stop words and lemmatization
    txt = [token.lemma_ for token in txt if not token.is_stop]

    return ' '.join(txt)

def n_grams(tokens, n):
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]


def generate_list_of_skills(text):
    nlp_text = nlp(text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    # all the resume skills will be saved here
    skillset = []
        
    # check for one-grams (example: python)
    for token in tokens:
        if nlp(token)[0].tag_ != 'VBN':
            skillset.append(token)
        
    # check for noun_chunks (example: machine learning)
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        skillset.append(token)

    # check for bigrams that SpaCy missed in the noun_chuncks
    for token in n_grams(tokens, 2):
        token = token.lower().strip()
        skillset.append(token)

    # check for trigrams that SpaCy missed in the noun_chuncks
    for token in n_grams(tokens, 3):
        token = token.lower().strip()
        skillset.append(token)

    return skillset

Skills Extraction from Resume Functions

In [ ]:
def extract_skills(resume_text, skills_api, clean=True):
    if clean == True:
        resume_text = preprocess(resume_text)
    
    skillset = set([i for i in set([i.lower() for i in generate_list_of_skills(resume_text)])])
    return skillset.intersection(skills_api)

def date_search(resume):
    ans = []

    # find all the date occurrence based on the regular expression
    pattern = r'(((Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Nov(ember)?|Dec(ember)?)|(\d{1,2}\s?\/){0,2}|(\d{1,2}\s?\-){0,2})\s?[-/ ]?\s?\d{4}?)|\bPresent\b|\bpresent\b|\bCurrent\b|\bcurrent\b'
    res = list(re.finditer(pattern, resume))

    if len(res) > 1:
        for ele in res:
            # this is to eradicate the results of having only year but without month
            if len(ele.group().strip()) > 5:
                ans.append([ele.start(), ele.end(), ele.group().strip()])

    res = []
    # Convert "present" and "current" to today's date
    for ele in ans:
        if ele[2].lower() == 'present' or ele[2].lower() == 'current':
            today = pd.to_datetime('today').date()
            ele[2] = today
            res.append(ele)
        else:
            # catch DateParse Error here
            try:
                day = pd.to_datetime(ele[2]).date()
                ele[2] = day
                res.append(ele)
            except :
                print('Cannot parse the date: ', ele[2])

    
    # all the date results are given in the form of [datetime_start_index, datetime_end_index, datetime]
    return res

def experience_tagging(date_list):
    i = 1
    cleaned_section = {}

    while i < len(date_list):

        prev = date_list[i-1]
        cur = date_list[i]
        
        if cur[0] < prev[1] + 10:
            # Taking ceiling of the year of experience
            key = ((cur[2] - prev[2]).days // 365) + 1
            
            # section starts at the (end index of the current date) + 1
            frm = cur[1]+1

            if i < len(date_list) - 1:
                # if there is another date that appears later, then the section will be until the start index of the next date
                until = date_list[i+1][0]
            else:
                # else the section will be until the end of the resume
                until = -1
            

            # Multiple projects with same year of experience, we do 'chaining' here
            if key in cleaned_section:
                cleaned_section[key].append((frm, until))
            else:
                cleaned_section[key] = [(frm, until)]
            i += 2

        else:

            # ignore the current date, possibly it is useless
            i += 1

    return cleaned_section

def skills_experience_level_identification(resume):
    res = {}
    date_list = date_search(resume)
    experience_sec = experience_tagging(date_list)

    for key in experience_sec:
        for start, end in experience_sec[key]:
            skills_list = extract_skills(resume[start:end], skills_api, True)
            for ele in skills_list:
                if ele not in res:
                    res[ele] = key
                else:
                    res[ele] = max(key, res[ele])

    skills_list = extract_skills(resume, skills_api, True)
    for ele in skills_list:
        if ele not in res:
            res[ele] = 1

    return dict(sorted(res.items(), key = lambda x:x[1], reverse = True))

Test Cases Output

In [ ]:
resume_skills = skills_experience_level_identification(test2)
resume_skills

{'management': 18,
 'netscaler': 18,
 'business continuity': 18,
 'active directory': 18,
 'capacity planning': 18,
 'planning': 18,
 'application deployment': 18,
 'firewall': 18,
 'reliability': 18,
 'r': 18,
 'reduction': 18,
 'disaster recovery': 18,
 'research': 18,
 'application development': 18,
 'web development': 3,
 'system administration': 3,
 'network support': 3,
 'group policy': 3,
 'troubleshooting': 1,
 'technical support': 1,
 'project management': 1,
 'installation': 1,
 'desktop support': 1,
 'ghost': 1,
 'citrix certified administrator': 1,
 'adware': 1,
 'spyware': 1,
 'adobe acrobat': 1,
 'b': 1,
 'information technology': 1}

## 4. Job Description Data Exploration

In [ ]:
pd.set_option('display.max_rows', 1000)
df2["category"].value_counts()

Registered Nurses                                                                               2513
Personal Care Aides                                                                              508
Social and Human Service Assistants                                                              415
Home Health Aides                                                                                393
Medical and Health Services Managers                                                             253
Physical Therapists                                                                              154
Clinical Nurse Specialists                                                                       154
Occupational Therapists                                                                          145
Pharmacy Technicians                                                                             135
Medical Assistants                                                                         

Skills Extraction from Job Description Function

In [ ]:
def extract_job_skills(job_texts, skills_api, clean=True):
  skillset_summary = {}

  for job_text in job_texts:
    if clean == True:
        job_text = preprocess(job_text)
    
    # get frequency in each document
    skillset = [i.lower() for i in generate_list_of_skills(job_text)]
    skillset_count = {}
    for skill in skillset:
      if skill in skills_api:
        if skill in skillset_count:
          skillset_count[skill] = skillset_count[skill] + 1
        else:
          skillset_count[skill] = 1

    
    for skill in skillset_count:
      '''
      # sum up frequency inside summary dictionary
      if skill in skillset_summary:
        skillset_summary[skill] = skillset_summary[skill] + skillset_count[skill]
      else:
        skillset_summary[skill] = skillset_count[skill]
      '''

      # append frequency inside summary dictionary 
      if skill in skillset_summary:
        skillset_summary[skill].append(skillset_count[skill])
      else:
        skillset_summary[skill] = [skillset_count[skill]]

  # get mean frequency
  for skill in skillset_summary:
    lst = skillset_summary[skill]
    skillset_summary[skill] = sum(lst)/len(lst)
  
  
  return dict(sorted(skillset_summary.items(), key = lambda x:x[1], reverse = True))

Test Cases

In [ ]:
job = "software developer"
job_texts = list(df2[df2["category"].str.lower().str.contains(job)]["job_requirements"])
print(job_texts,"\n")

job_skills = extract_job_skills(job_texts,skills_api,clean=True)
job_skills

## 5. Identifying Gaps among Candidates' Skills and Job's Required Skills

In [ ]:
courses = pd.read_excel('Courses.xlsx')
courses['Outcome Description'] = courses['jobFamily'] + " " \
                                + courses['Marketing Name'] + " " \
                                + courses['courseName'] + " " \
                                + courses ['moduleName'] + " " \
                                + courses['courseDesc'] + " " \
                                + courses['Outcome Description'] + " " \
                                + courses['competencyUnitDesc']
courses = courses[['productId', 'Marketing Name', 'Outcome Description', 'jobFamily', 'competencyLevel']]
courses['Outcome Description'] = courses['Outcome Description'].astype(str)
courses['Outcome Description'] = courses['Outcome Description'].apply(preprocess)

### 5.1 Gaps among Candidates' skills

In [ ]:
skill_freq_among_cand = pd.read_feather('output.feather')


In [4]:
resume = pd.read_csv('Resume.csv')
y = resume['Category']
skill_freq_among_cand['y'] = y
skill_freq_among_cand

,abaqus,ablation,ableton live,abnormal psychology,absorption,ac motors,academic advising,academic english,academic integrity,academic writing,...,zbrush,zemax,zendesk,zeta potential,zoning,zoology,zoom,zumba,zumba fitness,y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION


In [5]:
agg_data = skill_freq_among_cand.groupby(['y']).sum()
agg_data = agg_data.T

In [6]:
skills_required = {}
for col in agg_data.columns:
    print(col)
    series = agg_data[col][agg_data[col] > np.percentile(agg_data[col][agg_data[col] != 0], 95)]
    scaled_series = series.apply(lambda x: (x - series.min()) / (series.max() - series.min()))
    binned_series = scaled_series.apply(lambda x: 5 if x > 0.7 else 4 if x > 0.3 else 3)
    skills_required[col] = binned_series.to_dict()

ACCOUNTANT
ADVOCATE
AGRICULTURE
APPAREL
ARTS
AUTOMOBILE
AVIATION
BANKING
BPO
BUSINESS-DEVELOPMENT
CHEF
CONSTRUCTION
CONSULTANT
DESIGNER
DIGITAL-MEDIA
ENGINEERING
FINANCE
FITNESS
HEALTHCARE
HR
INFORMATION-TECHNOLOGY
PUBLIC-RELATIONS
SALES
TEACHER


In [ ]:
# skills_file = open('skills_required.pkl', 'wb')
# pickle.dump(skills_required, skills_file)
# skills_file.close()

In [2]:
with open('skills.pkl', 'rb') as pickle_file:
    skills = pickle.load(pickle_file)

with open('skills_required.pkl', 'rb') as pickle_file:
    skills_required = pickle.load(pickle_file)

In [3]:
# Map years of experience to competency level
def resume_level_deduction(years):
  if years <= 2:
    return 3
  elif years <= 5:
    return 4
  else:
    return 5
  
def skill_gap_identification_peers(skills, skills_required):
    diff = {}
    for key in skills_required:
        if key not in skills:
            diff[key] = [x for x in range(3, skills_required[key] + 1)]
        elif resume_level_deduction(skills[key]) < skills_required[key]:
            diff[key] = [x for x in range(resume_level_deduction(skills[key]), skills_required[key] + 1)]
    return diff

skill_gap_identification_peers(skills[31605080], skills_required['AVIATION'])

{'aircraft maintenance': [3],
 'aviation': [4, 5],
 'b': [3],
 'business administration': [3],
 'c': [3],
 'construction': [3],
 'critical thinking': [3],
 'drawing': [3],
 'filing': [3],
 'inquiry': [3],
 'inventory control': [3],
 'investigation': [3],
 'leadership': [3, 4],
 'm': [3],
 'management': [4, 5],
 'marketing': [3],
 'microsoft office': [3],
 'microsoft word': [3],
 'negotiation': [3],
 'planning': [3, 4],
 'process improvement': [3],
 'procurement': [3],
 'project management': [3],
 'purchasing': [3],
 'quality assurance': [3],
 'quality control': [3],
 'r': [3],
 'reduction': [3],
 'requisition': [3],
 'safety training': [3],
 'scheduling': [3],
 'security clearance': [3],
 'source': [3],
 'supervision': [3],
 'test equipment': [3],
 'time management': [3],
 'track': [3, 4],
 'tracking': [3]}

In [15]:
def tagcol_paragraph_embeddings_features(train_data):

    # Expects a dataframe with a 'Outcome Description' column
    train_data_values = train_data['Outcome Description'].values
    
    # Remember to use token.text to get the raw string, otherwise doc2vec cannot build vocabulary
    columns = [TaggedDocument([token.text for token in nlp(text) if token is not token.is_stop] , [i]) for i, text in enumerate(train_data_values)]
    
    return columns

In [ ]:
corpus = tagcol_paragraph_embeddings_features(courses)
model = Doc2Vec(dm=0, vector_size=50, workers=multiprocessing.cpu_count(), min_count=2, epochs=100, hs=1, negative=0)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
vector = model.infer_vector(
    ['marketing',
     'microsoft office',
     'microsoft word',
     'negotiation',
     'planning',
     'process improvement',
     'procurement',
     'project management',
     'purchasing',
     'quality assurance',
     'quality control'])
res = model.dv.most_similar([vector], topn=10)


In [ ]:
course_unique = set()
course_list = []
for i, prob in res:
    if courses.loc[i, 'Marketing Name'] not in course_unique:
        course_unique.add(courses.loc[i, 'Marketing Name'])
        course_list.append((courses.loc[i, 'productId'], courses.loc[i, 'Marketing Name'], courses.loc[i, 'competencyLevel']))
course_list

### 5.2 Gaps from Job Descriptions' skills

We use a function to place the candidate's skills and the skills of the required role into three levels "1, 2, 3". From there, we compare the levels between the candidate and the role and the output gives us a dictionary where the key is the skill and the level required for the candidate to achieve. 

In [ ]:
def resume_level_deduction(years):
  if years <= 2:
    return 3
  elif years <= 5:
    return 4
  else:
    return 5

# np.percentile(agg_data['ACCOUNTANT'][agg_data['ACCOUNTANT'] != 0], 95)
def job_level_deduction(count,max_count):
  if count < max_count * 0.05: # remove skill
    return 0
  elif count < max_count * 0.3: # entrant level
    return 3
  elif count < max_count * 0.7:  # specialist level
    return 4
  else:                      # expert level
    return 5

def extract_gap(job_skills, resume_skills):
  gap_skills = {}
  max_count = max(job_skills.values())
  for skill in job_skills:

    count = job_skills[skill]
    job_level = job_level_deduction(count,max_count)
    if job_level == 0:
      continue
    
    elif skill not in resume_skills: 
      gap_skills[skill] = [l for l in range(1,job_level+1)]

    else:
      resume_level = resume_level_deduction(resume_skills[skill])
      if resume_level >= job_level:
        continue
      else:
        gap_skills[skill] = [l for l in range(resume_level+1,job_level+1)]
  
  return dict(sorted(gap_skills.items(), key = lambda x:x[1], reverse = True))


In [ ]:
gap_skills = extract_gap(job_skills, resume_skills)
gap_skills

{'salesforce': [1, 2, 3],
 'snowflake': [1, 2],
 'signal processing': [1],
 'information architecture': [1],
 'geology': [1],
 'security clearance': [1],
 'splunk': [1],
 'sql': [1]}

In [ ]:
# Yung Jer's function
# Change format for dot2vec

Course Data Exploration

In [ ]:
pd.read_excel('/content/drive/MyDrive/BT4103/Courses_New.xlsx')

,productId,Marketing Name,courseName,courseDesc,Outcome Description,moduleName,competencyUnitCode,competencyUnitDesc,jobFamily,competencyLevel
0,203.0,Private Cloud,NICF - Private Cloud,This course is intended for information techno...,Learn how to manage and administer a cloud env...,Monitoring and Operating a Private Cloud,IT-CLA-503S-1,Design cloud infrastructure architecture (3 Cr...,Cloud Computing,5 - Expert Level
1,703.0,Business Process Engineering,NICF-Business Process Engineering (SF),This module examines the strategic and organiz...,Strategic and organizational issues of process...,NICF-Business Process Engineering (SF) (TGS-20...,ICT-SNA-5012-1.1,Enterprise architecture (2 Credit Values),Infrastructure Architecture,5 - Expert Level
2,704.0,Express BI,Express BI,This course provides the learner with knowledg...,Implement and administer SAP Net Weaver Busine...,NICF - Data Modelling & Transformation,IT-DE-502S-1,"Design extract, transform and load (ETL) proce...",Software Design & Development,5 - Expert Level
3,711.0,Advanced Digital Marketing (SF),NICF - Advanced Digital Marketing,Lithan’s Advanced Digital Marketing enables th...,"Learn digital marketing strategies, customer l...",NICF - Advanced Digital Marketing,IT-DIG-501S-1,Develop digital marketing campaign (3 Credit V...,Digital Advertising / Digital Distribution,5 - Expert Level
4,1305.0,Data Modelling & Transformation (SAP),NICF - Data Modelling and Transformation,This course&nbsp;of study provides Learners wi...,Learn how to extract master data and transacti...,NICF - Data Modelling & Transformation,IT-DE-502S-1,"Design extract, transform and load (ETL) proce...",Software Design & Development,5 - Expert Level
...,...,...,...,...,...,...,...,...,...,...
1175,15506.0,Bachelor Degree Software Engineering Politekni...,Bachelor Degree Software Engineering,This course is a work-study degree program des...,Learn front end programming skills such as HTM...,Front-End Web Development,App-Degree-FED,Front-End Web Development,Software Design & Development,3 - Entrant Level
1176,15601.0,Bachelor of Technology in Software Engineering...,Bachelor Degree of Software Engineering,This course is a work-study degree program des...,Learn front end programming skills such as HTM...,Front End Development,App-Degree-FED,Front-End Web Development,Software Design & Development,3 - Entrant Level
1177,15902.0,Bachelor of Technology in Software Engineering...,Bachelor of Technology in Software Engineering,This course is a work-study degree program des...,"Learn front-end development through HTML, CSS,...",Front End Development,App-Degree-FED,Front-End Web Development,Software Design & Development,3 - Entrant Level
1178,15903.0,Bachelor of Technology in Software Engineering...,Bachelor of Technology in Software Engineering,This course is a work-study degree program des...,"Learn front-end development through HTML, CSS,...",Front End Development,App-Degree-FED,Front-End Web Development,Software Design & Development,3 - Entrant Level


In [ ]:
pd.set_option('display.max_rows', 1000)
df3 = pd.read_excel('/content/drive/MyDrive/BT4103/Courses_New.xlsx')
df3["jobFamily"].value_counts()

Software Design & Development                 514
Infocomm Sales & Marketing                    195
Infrastructure Support                        105
Business Analytics                            105
Digital Advertising / Digital Distribution    100
Infrastructure Architecture                    37
Enterprise Mobility                            17
Project Management                             16
Strategy and Architecture                      15
Service Innovation Design                      14
Infocomm Security                              13
Sales and Marketing                            12
Cloud Computing                                 9
Data Centre Management                          9
Product Management                              7
IT Management                                   5
Enterprise Network Design Management            5
IT Outsourcing Management                       1
Generic Skills                                  1
Name: jobFamily, dtype: int64

In [ ]:
df3 = pd.read_excel('/content/drive/MyDrive/BT4103/Courses_New.xlsx')
# 'productId', 'competencyUnitCode',
# 'jobFamily', 'Marketing Name', 'courseName', 'moduleName',
# 'courseDesc', 'Outcome Description', 'competencyUnitDesc',
# 'competencyLevel'

# drop irrelevant columns
df3 = df3.drop(columns=['competencyUnitCode'])

# fill null values
df3 = df3.fillna('')

df3["ID"] = df3["productId"]
df3["name"] = df3['Marketing Name']
df3["text"] = " " + df3['jobFamily'] + " " + df3['Marketing Name'] + " " + df3['courseName'] + " " + df3 ['moduleName'] + " " + \
              " " + df3['courseDesc'] + " " + df3['Outcome Description'] + " " + df3['competencyUnitDesc'] + " "
df3["skills"] = df3['text'].apply(lambda x: extract_skills(x,skills_api,clean=True))
df3["text"] = df3["skills"].apply(" ".join)
df3["level"] = df3["competencyLevel"].apply(lambda x: 3 if "5" in x else (2 if "4" in x else 1))

df3 = df3[["ID","level","name","text","skills"]]
df3
#df3.to_csv("/content/drive/MyDrive/BT4103/Courses_Cleaned.csv",index=False)

In [ ]:
def parse_set_string(s):
    if s == '{}':
        return set()
    else:
        words = re.sub('[^a-z\s,]','', s)  
        words_list = words.split(',')
        return set(map(lambda x: str(x).strip(), words_list))

df3 = pd.read_csv("/content/drive/MyDrive/BT4103/Courses_Cleaned.csv",
                  dtype = {'ID': int, 'level': int, 'name': str, 'text': str, 'skills': str})
df3.skills = df3.skills.apply(parse_set_string)
df3

,ID,level,name,text,skills
0,203,3,Private Cloud,information technology r cloud infrastructure ...,"{cloud infrastructure, infrastructure architec..."
1,703,3,Business Process Engineering,enterprise architecture enterprise resource pl...,"{management, process design, infrastructure ar..."
2,704,3,Express BI,software design warehousing extract transform ...,"{extract transform load, functional specificat..."
3,711,3,Advanced Digital Marketing (SF),session dashboard digital advertising digital ...,"{dashboard, session, marketing, digital market..."
4,1305,3,Data Modelling & Transformation (SAP),sap netweaver software design dimensional anal...,"{dimensional analysis, business process, extra..."
...,...,...,...,...,...
1175,15506,1,Bachelor Degree Software Engineering <p> Polit...,software design prototype software development...,"{hibernate, reinforcement, deep learning, soft..."
1176,15601,1,Bachelor of Technology in Software Engineering...,software design prototype software development...,"{hibernate, reinforcement, deep learning, soft..."
1177,15902,1,Bachelor of Technology in Software Engineering...,prototype software design software development...,"{reinforcement, deep learning, software design..."
1178,15903,1,Bachelor of Technology in Software Engineering...,prototype software design software development...,"{reinforcement, deep learning, software design..."


In [ ]:
import time
start_time = time.time()

path = {}

'''
for skill in gap_skills:
# get courses from 3 most similar families
  # df3_group = df3.copy().groupby(['family']).agg(list).reset_index()[["family"]]
  # df3_group["similarity"] = df3_group["family"].apply(lambda x: nlp(skill).similarity(nlp(x)))
  # family = df3_group.nlargest(3,'similarity',keep='all')["family"].values
  # df3_family = df3.copy()
  # df3_family = df3_family[df3_family["family"].isin(family)]

  # get 10 courses with most similar names
  # df3_level["similarity"] = df3_level["name"].apply(lambda x: nlp(skill).similarity(nlp(x)))
  # df3_level = df3_level.nlargest(10,'similarity',keep='all')
'''

for skill in gap_skills:

  # get courses that mention skill
  df3_A = df3.copy()
  df3_A = df3_A[df3_A["skills"].apply(lambda x: skill in x)]

  for level in gap_skills[skill]:

    # get courses of same competency level
    df3_B = df3_A.copy()
    df3_B = df3_B[df3_B["level"].astype(int) == int(level)]

    if df3_B.empty:
      courses = []
      print("No Courses for " + skill + " at level " + str(level) + ".")
    else:
      # get similarity score
      df3_B["similarity"] = df3_B["text"].apply(lambda x: nlp(skill).similarity(nlp(x)))
      top_courses = df3_B.nlargest(3,'similarity',keep='all')
      courses = top_courses["name"].values
      #courses = [(name[:40] + '..') if len(name) > 40 else name for name in courses]
    
    # update path
    for course in courses:
      if level not in path:
        path[level] = {}
      if course not in path[level]:
        path[level][course] = [skill]
      elif skill not in path[level][course]:
        path[level][course].append(skill)

path = dict(sorted(path.items(), key = lambda x:len(x), reverse = True))
for x in path:
  print(x,path[x])

print("--- %s seconds ---" % (time.time() - start_time))

No Courses for salesforce at level 1.
No Courses for salesforce at level 3.
No Courses for snowflake at level 1.
No Courses for snowflake at level 2.
No Courses for signal processing at level 1.
No Courses for information architecture at level 1.
No Courses for geology at level 1.
No Courses for security clearance at level 1.
No Courses for splunk at level 1.


<ipython-input-51-bd41ad70fddd>:37: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  df3_B["similarity"] = df3_B["text"].apply(lambda x: nlp(skill).similarity(nlp(x)))


2 {'Express CRM (Salesforce)': ['salesforce'], 'Express CRM (Salesforce) <p> for Non-technical Professionals': ['salesforce'], 'Express CRM <p> (with Salesforce CRM) ': ['salesforce']}
1 {'Operational Analytics': ['sql'], 'Express Web Developer (PHP)': ['sql'], 'Business Analytics': ['sql']}
--- 0.25696778297424316 seconds ---


In [ ]:
print(nlp("management").similarity(nlp("cyber security management")))
print(nlp("cyber security management").similarity(nlp("management")))

0.518793223099013
0.518793223099013


<ipython-input-20-b33acc2765ab>:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(nlp("management").similarity(nlp("cyber security management")))
<ipython-input-20-b33acc2765ab>:2: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vec